# Predictive Modeling of Hospital Length of Stay and Discharge Type
# [Step 3: Merge laboratory and clinical data]

This notebook merges the laboratory results dataset with the clinical dataset and performs additional data cleaning.  

**Libraries used:**  
- pandas  
- matplotlib  
- seaborn  
- simple_icd_10  

**Last updated:** March 26, 2025.  

## 1. Import libraries and load datasets

In [1]:
# Import data manipulation library
import pandas as pd

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
# Define file paths for cleaned input datasets  
lab_data_path = "/home/anna/Desktop/Master_thesis/output_data/pre-cleaned_lab_data" 
clinical_data_path = "/home/anna/Desktop/Master_thesis/output_data/cleaned_clinical_data"  

# Define output path for merged dataset  
merged_data_path = "/home/anna/Desktop/Master_thesis/output_data/2_merged_data"  

In [3]:
# Load lab dataset 
lab_data = pd.read_csv(lab_data_path)

In [4]:
display(lab_data)

,patient_id,case_id,test_name,test_abbr,method_number,numeric_result,text_result,unit
0,1,171465,Natrium,Na,1,138.0,138,mmol/L
1,1,171465,Kalium,KA,3,4.6,4.6,mmol/L
2,1,171465,Hämolytisch,H-Se,42,4.0,4,Unknown
3,1,171465,Lipämisch,L-Se,43,3.0,3,Unknown
4,1,171465,Ikterisch,I-Se,44,1.0,1,Unknown
...,...,...,...,...,...,...,...,...
19313528,240990,415184,MCHC,MCHCn,9116,333.0,333,g/L
19313529,240990,415184,RDW,RDWn,9117,14.3,14.3,%
19313530,240990,415184,Thrombozyten,THZn,9119,226.0,226,G/L
19313531,240990,415184,MPV,MPVn,9204,9.0,9.0,fL


In [5]:
# Load and display clinical dataset
clinical_data = pd.read_csv(clinical_data_path)
display(clinical_data)

,patient_id,case_id,discharge_type,sex,age,length_of_stay_days,diagnosis,diagnosis_category
0,1,171465,Entlassung,f,73,28,A04.70,Infectious diseases
1,1,333396,Entlassung,f,73,34,K57.22,Digestive diseases
2,2,27091,Entlassung,m,51,13,T84.5,Injury & poisoning
3,2,36154,Entl.ext.Instit,m,50,12,I21.4,Circulatory diseases
4,2,142617,Entl. in ex.KH,m,51,1,T84.5,Injury & poisoning
...,...,...,...,...,...,...,...,...
273558,240988,393440,Entlassung,m,79,2,K55.21,Digestive diseases
273559,240988,412516,Entlassung,m,79,10,I11.00,Circulatory diseases
273560,240988,425806,Entl.ext.Instit,m,79,7,M16.7,Musculoskeletal diseases
273561,240989,393141,Entlassung,m,62,1,I49.3,Circulatory diseases


In [6]:
# Display unique entries for lab_data
print(lab_data.nunique())

patient_id        182214
case_id           311547
test_name           2893
test_abbr           3891
method_number       3978
numeric_result     62295
text_result        98346
unit                 167
dtype: int64


## 2. Filter out lab tests with more than 80% missing data  

In [7]:
# Select most frequent lab tests based on case occurrence

# Count the number of unique cases per lab test
lab_test_counts = lab_data.groupby("test_abbr", as_index=False)["case_id"].nunique()
lab_test_counts.rename(columns={"case_id": "num_cases"}, inplace=True)

# Get total number of unique cases
total_cases = lab_data["case_id"].nunique()

# Calculate missing percentage as cases that do NOT have the test
lab_test_counts["missing_percentage"] = ((total_cases - lab_test_counts["num_cases"]) / total_cases) * 100

# Sort lab tests by frequency (most frequent first)
lab_test_counts.sort_values(by="num_cases", ascending=False, inplace=True)

# Filter lab tests that have less than 80% missing data
filtered_lab_tests = lab_test_counts[lab_test_counts["missing_percentage"] < 80][["test_abbr", "missing_percentage"]]

In [8]:
print(filtered_lab_tests.to_string(index=False))

test_abbr  missing_percentage
       KA           14.160945
    Leukn           15.383875
      Hbn           15.384838
     Eryn           15.386442
      Hkn           15.387405
     THZn           15.389010
    MCHCn           15.389331
     MCHn           15.389652
     MCVn           15.389652
     RDWn           15.462194
     MPVn           16.124694
       Na           16.194346
        L           18.774374
        I           18.774695
        H           18.775658
       GL           22.095222
   NRBCmn           23.696906
      CRP           24.642189
       CR           25.421205
    INRiH           25.544781
     QUHD           25.549917
   ENTN1n           29.524277
   EPIGFR           30.738540
   Quicks           36.442977
     ASAT           61.684433
     ALAT           62.081163
   Ben-ID           64.573564
      GGT           65.179251
     UREA           65.342629
       CA           71.262121
    EC3-U           71.551002
    GLUC3           71.551965
    PH4-U 

### 2.1 Removal of Specific Lab Tests  

Excluded tests fall into two categories:  

1. **Metadata Entries** (non-result data)  
2. **Highly Variable Urinary Tests** (fluctuate reducing reliability)  

In [9]:
remove_tests = [
    "EC3-U", "PH4-U", "BI3-U", "LK3-U", "PROT3", "NITR3", "KETO3", "SPEZ3",
    "FARBE3", "TRUEB3", "UST1", "BA-Ux", "LK-Ux", "KRI-Ux", "ERY-Ux", "PI-Ux", "EART",
    "GLUC3", "URO3", "ENTN1n", "EART", "TNT_hn", "Ben-ID"
]

# Remove specified lab tests from the filtered list  
filtered_lab_tests = filtered_lab_tests[~filtered_lab_tests["test_abbr"].isin(remove_tests)]

In [10]:
display(filtered_lab_tests)

# Print the number of rows (tests) in filtered_lab_tests
print(f"Number of tests in filtered_lab_tests: {filtered_lab_tests.shape[0]}")

,test_abbr,missing_percentage
1747,KA,14.160945
2008,Leukn,15.383875
1627,Hbn,15.384838
1198,Eryn,15.386442
1640,Hkn,15.387405
2982,THZn,15.389010
2064,MCHCn,15.389331
2066,MCHn,15.389652
2069,MCVn,15.389652
2623,RDWn,15.462194


Number of tests in filtered_lab_tests: 49


In [11]:
# Filter the lab_data to keep only rows where test_abbr is in the filtered_lab_tests list
lab_data_filtered = lab_data[lab_data['test_abbr'].isin(filtered_lab_tests['test_abbr'])]

# Display the first few rows of the filtered data
lab_data_filtered.head()

,patient_id,case_id,test_name,test_abbr,method_number,numeric_result,text_result,unit
0,1,171465,Natrium,Na,1,138.0,138,mmol/L
1,1,171465,Kalium,KA,3,4.6,4.6,mmol/L
2,1,171465,Hämolytisch,H-Se,42,4.0,4,Unknown
3,1,171465,Lipämisch,L-Se,43,3.0,3,Unknown
4,1,171465,Ikterisch,I-Se,44,1.0,1,Unknown


In [12]:
# Free up memory
del lab_data

## 3. Merge the two data sets

In [13]:
# Convert lab data to wide format, keeping only the first test result per patient-case
lab_data_wide = lab_data_filtered.pivot_table(
    index=['patient_id', 'case_id'],  # Keep patient_id and case_id as index
    columns='test_abbr',  # Pivot on 'test_abbr'
    values='numeric_result',  # Use numeric_result as values
    aggfunc='first' # First measurement = Initial test at admission
).reset_index()

# Display the first few rows of the wide-format data
lab_data_wide.head()

test_abbr,patient_id,case_id,ALAT,AP,ASAT,BASm#n,BIg,CA,CK,CO-HB,...,Quicks,RDWn,THZn,TNThsn,Tbga,UREA,pCO2,pH,pO2,tHb
0,1,171465,10.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN,...,13.2,18.3,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,333396,19.0,93.0,22.0,0.19,8.0,2.02,NaN,NaN,...,11.6,18.8,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,27091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,18.9,164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,36154,38.0,58.0,NaN,NaN,NaN,NaN,347.0,NaN,...,12.7,14.4,131.0,367.0,NaN,4.5,NaN,NaN,NaN,NaN
4,2,142617,15.0,65.0,19.0,0.04,16.0,2.39,NaN,NaN,...,19.2,17.6,159.0,NaN,NaN,5.2,NaN,NaN,NaN,NaN


In [14]:
# Merge lab_data with clinical_data on patient_id and case_id
merged_data = pd.merge(clinical_data, lab_data_wide, on=['patient_id', 'case_id'], how='inner')

In [15]:
# Check for duplicate rows
duplicates = merged_data.duplicated()

# Count total duplicate rows
num_duplicates = duplicates.sum()
print(f"Total duplicate rows: {num_duplicates}")

# Display some duplicate rows (if they exist)
if num_duplicates > 0:
    display(merged_data[duplicates].head())

Total duplicate rows: 0


In [16]:
display(merged_data)

,patient_id,case_id,discharge_type,sex,age,length_of_stay_days,diagnosis,diagnosis_category,ALAT,AP,...,Quicks,RDWn,THZn,TNThsn,Tbga,UREA,pCO2,pH,pO2,tHb
0,1,171465,Entlassung,f,73,28,A04.70,Infectious diseases,10.0,NaN,...,13.2,18.3,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,333396,Entlassung,f,73,34,K57.22,Digestive diseases,19.0,93.0,...,11.6,18.8,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,27091,Entlassung,m,51,13,T84.5,Injury & poisoning,NaN,NaN,...,14.0,18.9,164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,36154,Entl.ext.Instit,m,50,12,I21.4,Circulatory diseases,38.0,58.0,...,12.7,14.4,131.0,367.0,NaN,4.5,NaN,NaN,NaN,NaN
4,2,142617,Entl. in ex.KH,m,51,1,T84.5,Injury & poisoning,15.0,65.0,...,19.2,17.6,159.0,NaN,NaN,5.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269015,240988,393440,Entlassung,m,79,2,K55.21,Digestive diseases,12.0,NaN,...,10.3,13.8,176.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269016,240988,412516,Entlassung,m,79,10,I11.00,Circulatory diseases,11.0,NaN,...,11.0,14.3,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269017,240988,425806,Entl.ext.Instit,m,79,7,M16.7,Musculoskeletal diseases,NaN,NaN,...,NaN,13.2,219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269018,240989,393141,Entlassung,m,62,1,I49.3,Circulatory diseases,NaN,NaN,...,10.9,13.6,269.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5. Translate discharge type

In [17]:
merged_data['discharge_type'].unique()

array(['Entlassung', 'Entl.ext.Instit', 'Entl. in ex.KH',
       'Entl. Tarifbere', 'Verstorben', 'Entl.eigner Wun',
       'Rückv n. ambBeh', 'Wartepat. Entl.', 'Ambulanz exInst',
       'Besuch Krankh.'], dtype=object)

In [18]:
# Mapping of discharge types from German to English
discharge_translation = {
    "Entlassung": "Discharge",
    "Entl.ext.Instit": "Discharge to another institution",
    "Entl. in ex.KH": "Discharge to another hospital",
    "Verstorben": "Deceased",
    "Entl. Tarifbere": "Discharge tariff status",
    "Entl.eigner Wun": "Discharge on patient’s own request",
    "Wartepat. Entl.": "Discharge of a waiting patient",
    "Ambulanz exInst": "Ambulance to external institution",
    "Rückv n. ambBeh": "Transfer back from another institution",
    "Besuch Krankh.": "Visit to the hospital"
}


# Translate the discharge type column in merged_data
merged_data['discharge_type'] = merged_data['discharge_type'].map(discharge_translation)

In [19]:
merged_data.head()

,patient_id,case_id,discharge_type,sex,age,length_of_stay_days,diagnosis,diagnosis_category,ALAT,AP,...,Quicks,RDWn,THZn,TNThsn,Tbga,UREA,pCO2,pH,pO2,tHb
0,1,171465,Discharge,f,73,28,A04.70,Infectious diseases,10.0,NaN,...,13.2,18.3,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,333396,Discharge,f,73,34,K57.22,Digestive diseases,19.0,93.0,...,11.6,18.8,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,27091,Discharge,m,51,13,T84.5,Injury & poisoning,NaN,NaN,...,14.0,18.9,164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,36154,Discharge to another institution,m,50,12,I21.4,Circulatory diseases,38.0,58.0,...,12.7,14.4,131.0,367.0,NaN,4.5,NaN,NaN,NaN,NaN
4,2,142617,Discharge to another hospital,m,51,1,T84.5,Injury & poisoning,15.0,65.0,...,19.2,17.6,159.0,NaN,NaN,5.2,NaN,NaN,NaN,NaN


## 5. Mapping ICD Codes to Descriptions

NOTE: Switzerland uses ICD-10-GM. The **simple_icd_10** library is currently used, but some codes are missing. Consider wrapping the ICD10gm R package for use in Python.

In [20]:
# Import the simple_icd_10 library for ICD-10 code descriptions  
import simple_icd_10 as icd

In [21]:
# Rename column
merged_data.rename(columns={'diagnosis': 'diagnosis_code'}, inplace=True)

def translate_icd(code):
    try:
        return icd.get_description(code)  # Try full code first
    except ValueError:
        if "." in code:  # If code has a dot, try the first part
            base_code = code.split(".")[0]
            try:
                return icd.get_description(base_code)
            except ValueError:
                return None  # Still not found
        return None  # Return None if no dot and not found

# Apply the modified function
merged_data["diagnosis_text"] = merged_data["diagnosis_code"].apply(translate_icd)

# Show codes that are still missing
missing_diagnosis_text = merged_data[merged_data['diagnosis_text'].isna()]
print(missing_diagnosis_text['diagnosis_code'].unique())

['T89.03' 'B59' 'T89.02' 'I84.8' 'T89.01' 'I84.1' 'D90' 'I84.9' 'U10.9']


## 6. Check plausibility of numeric_result values

### 6.1 Check negative entries

In [22]:
# Select only numeric columns (excluding categorical ones)
numeric_cols = merged_data.select_dtypes(include=['number'])

# Find columns with negative values
negative_values = (numeric_cols < 0).any()

# Display columns that contain negative values
negative_cols = negative_values[negative_values].index.tolist()
print("Lab test columns with negative values:", negative_cols)

Lab test columns with negative values: ['ALAT', 'ASAT', 'BIg', 'CA', 'CO-HB', 'CR', 'CRP', 'H', 'I', 'MTHB', 'QUHD']


In [23]:
# Show rows where negative values occur
df_negatives = merged_data[merged_data[negative_cols].lt(0).any(axis=1)]
print(df_negatives)

        patient_id  case_id                    discharge_type sex  age  \
141            127   193819                         Discharge   f   43   
1306          1197     4579                         Discharge   f   57   
1440          1300    54843     Discharge to another hospital   m   29   
3010          2739   308716                         Discharge   f   89   
3888          3629   185818     Discharge to another hospital   m   66   
...            ...      ...                               ...  ..  ...   
261622      237506   407781                         Discharge   m   82   
263487      238370   406179                         Discharge   m   50   
264190      238696   405485                         Discharge   f   76   
265989      239582   426330     Discharge to another hospital   f   40   
268999      240980   395983  Discharge to another institution   m   66   

        length_of_stay_days diagnosis_code        diagnosis_category  ALAT  \
141                       2      

In [24]:
merged_data[negative_cols] = merged_data[negative_cols].mask(merged_data[negative_cols] < 0, None)

### 6.2 Handle Outliers

Outlier handling is performed in a separate notebook: 4_outlier_analysis.ipynb.

## 7. Save merged dataset

In [27]:
merged_data.to_csv(merged_data_path, index=False)